# Introduction Resampling Methods

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [3]:
auto = sm.datasets.get_rdataset('Auto', 'ISLR')

In [4]:
print(auto.__doc__)

==== ===============
Auto R Documentation
==== ===============

Auto Data Set
-------------

Description
~~~~~~~~~~~

Gas mileage, horsepower, and other information for 392 vehicles.

Usage
~~~~~

::

   Auto

Format
~~~~~~

A data frame with 392 observations on the following 9 variables.

``mpg``
   miles per gallon

``cylinders``
   Number of cylinders between 4 and 8

``displacement``
   Engine displacement (cu. inches)

``horsepower``
   Engine horsepower

``weight``
   Vehicle weight (lbs.)

``acceleration``
   Time to accelerate from 0 to 60 mph (sec.)

``year``
   Model year (modulo 100)

``origin``
   Origin of car (1. American, 2. European, 3. Japanese)

``name``
   Vehicle name

The orginal data contained 408 observations but 16 observations with
missing values were removed.

Source
~~~~~~

This dataset was taken from the StatLib library which is maintained at
Carnegie Mellon University. The dataset was used in the 1983 American
Statistical Association Exposition.

References

In [98]:
# data
dauto = auto.data

### 1st Split - MSE values

In [6]:
from sklearn.model_selection import train_test_split

np.random.seed(1)

X = dauto['horsepower']
y = dauto['mpg']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.5)

#### Simple Linear Regression horsepower onto mpg

In [7]:
lr = sm.OLS(y_train, sm.add_constant(X_train)).fit()
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     280.5
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           1.54e-39
Time:                        10:04:04   Log-Likelihood:                -586.12
No. Observations:                 196   AIC:                             1176.
Df Residuals:                     194   BIC:                             1183.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.5927      1.014     39.037      0.0

In [8]:
# lr = LinearRegression()
# lr.fit(x_train.to_frame(),y_train)
# pred = lr.predict(x_test.to_frame())
# print('Mean squared error is ',mean_squared_error(y_test,pred))

from sklearn.metrics import mean_squared_error

y_preds = lr.predict(sm.add_constant(X_test))

print(f"MSE: {mean_squared_error(y_test,y_preds)}")

MSE: 24.80212062059356


#### Simple Linear Regression horsepower onto mpg (using polynomial 2)

In [9]:
def ortho_poly_fit(x, degree = 1):
    n = degree + 1
    x = np.asarray(x).flatten()
    if(degree >= len(np.unique(x))):
            stop("'degree' must be less than number of unique points")
    xbar = np.mean(x)
    x = x - xbar
    X = np.fliplr(np.vander(x, n))
    q,r = np.linalg.qr(X)

    z = np.diag(np.diag(r))
    raw = np.dot(q, z)

    norm2 = np.sum(raw**2, axis=0)
    alpha = (np.sum((raw**2)*np.reshape(x,(-1,1)), axis=0)/norm2 + xbar)[:degree]
    Z = raw / np.sqrt(norm2)
    return Z, norm2, alpha


X_train2 = sm.add_constant(ortho_poly_fit(X_train.values,2)[0][:,1:])
y_train2 = y_train

lr2 = sm.OLS(y_train2, X_train2).fit()
print(lr2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     184.7
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           1.49e-45
Time:                        10:04:06   Log-Likelihood:                -568.96
No. Observations:                 196   AIC:                             1144.
Df Residuals:                     193   BIC:                             1154.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.6214      0.317     74.408      0.0

In [10]:
y_preds2 = lr2.predict(sm.add_constant(ortho_poly_fit(X_test.values,2)[0][:,1:]))

print(f"MSE: {mean_squared_error(y_test,y_preds2)}")

MSE: 18.94779122522533


### Simple Linear Regression horsepower onto mpg (using polynomial 3)

In [11]:
X_train3 = sm.add_constant(ortho_poly_fit(X_train.values,3)[0][:,1:])
y_train3 = y_train

lr3 = sm.OLS(y_train3, X_train3).fit()
print(lr3.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                     122.6
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           2.23e-44
Time:                        10:04:07   Log-Likelihood:                -568.93
No. Observations:                 196   AIC:                             1146.
Df Residuals:                     192   BIC:                             1159.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.6214      0.318     74.227      0.0

In [12]:
y_preds3 = lr3.predict(sm.add_constant(ortho_poly_fit(X_test.values,3)[0][:,1:]))

print(f"MSE: {mean_squared_error(y_test,y_preds3)}")

MSE: 18.90233627602706


### 2nd Split - MSE values

In [13]:
from sklearn.model_selection import train_test_split

np.random.seed(2)

X1 = dauto['horsepower']
y1 = dauto['mpg']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1,
                                                   y1,
                                                   test_size = 0.5)

#### Simple Linear Regression horsepower onto mpg

In [14]:
lr11 = sm.OLS(y1_train, sm.add_constant(X1_train)).fit()
print(lr11.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     308.5
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           5.83e-42
Time:                        10:04:08   Log-Likelihood:                -592.23
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         40.8582      1.036     39.431      0.0

In [15]:
y1_preds = lr11.predict(sm.add_constant(X1_test))

print(f"MSE: {mean_squared_error(y1_test,y1_preds)}")

MSE: 23.442643969985753


#### Simple Linear Regression horsepower onto mpg (using polynomial 2)

In [16]:
X1_train2 = sm.add_constant(ortho_poly_fit(X1_train.values,2)[0][:,1:])
y1_train2 = y1_train

lr12 = sm.OLS(y1_train2, X1_train2).fit()
print(lr12.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     219.4
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           1.99e-50
Time:                        10:04:08   Log-Likelihood:                -569.29
No. Observations:                 196   AIC:                             1145.
Df Residuals:                     193   BIC:                             1154.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.7689      0.318     74.747      0.0

In [17]:
y1_preds2 = lr12.predict(sm.add_constant(ortho_poly_fit(X1_test.values,2)[0][:,1:]))

print(f"MSE: {mean_squared_error(y1_test,y1_preds2)}")

MSE: 18.93937268206552


#### Simple Linear Regression horsepower onto mpg (using polynomial 3)

In [18]:
X1_train3 = sm.add_constant(ortho_poly_fit(X1_train.values,3)[0][:,1:])
y1_train3 = y1_train

lr13 = sm.OLS(y1_train3, X1_train3).fit()
print(lr13.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     145.5
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           3.30e-49
Time:                        10:04:09   Log-Likelihood:                -569.28
No. Observations:                 196   AIC:                             1147.
Df Residuals:                     192   BIC:                             1160.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.7689      0.319     74.557      0.0

In [19]:
y1_preds3 = lr13.predict(sm.add_constant(ortho_poly_fit(X1_test.values,3)[0][:,1:]))

print(f"MSE: {mean_squared_error(y1_test,y1_preds3)}")

MSE: 18.981572123945305


### Leave-One-Out Cross-Validation

In [20]:
from sklearn.model_selection import train_test_split

np.random.seed(1)

X = dauto['horsepower']
y = dauto['mpg']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.5)

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()


loo.get_n_splits(X)



crossvalidation = KFold(n_splits=392, random_state=None, shuffle=False)
lr = LinearRegression()
lr.fit(X_train.to_frame(),y_train)
pred = lr.predict(X_test.to_frame())


scores = cross_val_score(lr, X.to_frame(), y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)

print("MSE: " + str(np.mean(np.abs(scores))))

MSE: 24.23151351792923


In [22]:
from sklearn.preprocessing import PolynomialFeatures

for i in range(1,6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X.to_frame())
    model = lr.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    
    print(" polynomial " +str(i)+ " MSE: " + str(np.mean(np.abs(scores)))+ ", STD: " + str(np.std(scores)))

 polynomial 1 MSE: 24.231513517929226, STD: 36.797315036405344
 polynomial 2 MSE: 19.248213124489787, STD: 34.99844615178275
 polynomial 3 MSE: 19.334984064101494, STD: 35.76513567801889
 polynomial 4 MSE: 19.424430308672896, STD: 35.68335275574849
 polynomial 5 MSE: 19.033217373107604, STD: 35.31732327822555


## K-fold

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

np.random.seed(1)

X = dauto['horsepower'].to_frame()
y = dauto['mpg']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.5)
lr = LinearRegression()
crossvalidation = KFold(n_splits=10, random_state=None, shuffle=False)

for i in range(1,11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lr.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    
    print(" polynomial " +str(i)+ " MSE: " + str(np.mean(np.abs(scores)))+ ", STD: " + str(np.std(scores)))

 polynomial 1 MSE: 27.439933652339864, STD: 14.51025071128113
 polynomial 2 MSE: 21.23584005580157, STD: 11.797327528897423
 polynomial 3 MSE: 21.336606183492073, STD: 11.844339714946903
 polynomial 4 MSE: 21.3538869961286, STD: 11.98633234961832
 polynomial 5 MSE: 20.905603446050055, STD: 12.185477989682207
 polynomial 6 MSE: 20.78789280660235, STD: 12.145705434704288
 polynomial 7 MSE: 20.953757892685807, STD: 12.059643641417342
 polynomial 8 MSE: 21.077118806792964, STD: 12.04446801214061
 polynomial 9 MSE: 21.036937754093266, STD: 11.948358265914973
 polynomial 10 MSE: 20.985037880274767, STD: 11.80440968886917


### The Bootstrap

In [24]:
portfolio = sm.datasets.get_rdataset('Portfolio', 'ISLR')
print(portfolio.__doc__)

========= ===============
Portfolio R Documentation
========= ===============

Portfolio Data
--------------

Description
~~~~~~~~~~~

A simple simulated data set containing 100 returns for each of two
assets, X and Y. The data is used to estimate the optimal fraction to
invest in each asset to minimize investment risk of the combined
portfolio. One can then use the Bootstrap to estimate the standard error
of this estimate.

Usage
~~~~~

::

   Portfolio

Format
~~~~~~

A data frame with 100 observations on the following 2 variables.

``X``
   Returns for Asset X

``Y``
   Returns for Asset Y

Source
~~~~~~

Simulated data

References
~~~~~~~~~~

James, G., Witten, D., Hastie, T., and Tibshirani, R. (2013) *An
Introduction to Statistical Learning with applications in R*,
`www.StatLearning.com <www.StatLearning.com>`__, Springer-Verlag, New
York

Examples
~~~~~~~~

::

   summary(Portfolio)
   attach(Portfolio)
   plot(X,Y)



In [25]:
# data
dportfolio = portfolio.data

In [26]:
def alpha(data, index):
    X = data['X'].loc[index]
    Y = data['Y'].loc[index]
    return ((np.var(Y)-np.cov(X,Y)[0][1])/(np.var(X)+np.var(Y)-2*np.cov(X,Y)[0][1]))

In [27]:
alpha(dportfolio,np.arange(0,100))

0.5766511516104116

In [28]:
from sklearn.utils import resample

i = resample(np.arange(0,100), n_samples=100, random_state=2, replace=True)
alpha(dportfolio,i)


0.6491078066222176

In [29]:
def boot(data,func,R):
    estimates = []
    for i in range(R):
        estimates.append(func(data,resample(np.arange(0,len(data)), n_samples=len(data), replace=True)))
    df = pd.DataFrame(estimates)
    bootstrap_statistics = {'estimated_value':np.mean(np.array(df.values),axis=0),'std_error':np.std(np.array(df.values),axis=0)}   
    return bootstrap_statistics

In [30]:
results = boot(dportfolio,alpha,1000)
results

{'estimated_value': array([0.58009862]), 'std_error': array([0.0915255])}

### Estimating the Accuracy of a Linear Regression Model

In [176]:
def boot_fn(data,index):
    X = data['horsepower'].loc[index]
    y = data['mpg'].loc[index]
    lr = LinearRegression()
    lr.fit(X.to_frame(),y)
    intercept = lr.intercept_
    coef = lr.coef_
    return [intercept,coef[0]]

In [177]:
dauto1 = dauto.reset_index()
print(boot_fn(dauto1,resample(np.arange(0,392), n_samples=100, replace=True)))

[40.40042993121894, -0.15927786906064584]


In [179]:
np.random.seed(1)
print(boot_fn(dauto1,resample(np.arange(0,392), n_samples=100, replace=True)))

[39.693439525976096, -0.15423909195849003]


In [180]:
print(boot_fn(dauto1,resample(np.arange(0,392), n_samples=100, replace=True)))

[39.15000955774242, -0.15116600864364851]


In [181]:
np.random.seed(1)
boot(dauto1,boot_fn,1000)

{'estimated_value': array([39.96338479, -0.1583057 ]),
 'std_error': array([0.82526625, 0.00713573])}

In [182]:
np.random.seed(1)

X = dauto1['horsepower'].to_frame()
y = dauto1['mpg']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.5)
lr = sm.OLS(y_train, sm.add_constant(X_train)).fit()
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     280.5
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           1.54e-39
Time:                        10:42:11   Log-Likelihood:                -586.12
No. Observations:                 196   AIC:                             1176.
Df Residuals:                     194   BIC:                             1183.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.5927      1.014     39.037      0.0

#### polynomial horse with bootstrap

In [189]:
def boot_fn(data,index):
    
    x = data['horsepower'].to_frame()
    poly = PolynomialFeatures(degree=2)
    X_current = poly.fit_transform(x)[index]
    y = data['mpg'].loc[index]
    lr = LinearRegression()
    lr.fit(X_current, y)
    intercept = lr.intercept_
    coef =lr.coef_
    
    return [intercept,coef[1], coef[2]]

In [190]:
from sklearn.utils import resample
boot(dauto1,boot_fn,1000)

{'estimated_value': array([ 5.68082795e+01, -4.64974801e-01,  1.22711212e-03]),
 'std_error': array([2.09757065e+00, 3.35744253e-02, 1.21825719e-04])}

In [191]:
x = pd.DataFrame(dauto1['horsepower'])
poly = PolynomialFeatures(degree=2)
X_current = poly.fit_transform(x)
y = dauto1['mpg']
lr = sm.OLS(y,X_current).fit()
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Mon, 08 Feb 2021   Prob (F-statistic):           5.40e-99
Time:                        10:45:44   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         56.9001      1.800     31.604      0.0